# 6 Chroma를 활용한 RAG 파이프라인 구축

In [1]:
! uv add -q python-dotenv langchain langchain-openai langchain-community langchain-text-splitters docx2txt langchain-chroma langchain-upstage

# 6-1 데이터 전처리를 위한 텍스트 분할기 설정     

In [3]:
# 문서 로더와 텍스트 분할기 임포트
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 텍스트 분할기 설정 - RAG에서 중요한 전처리 단계
text_splitter = RecursiveCharacterTextSplitter(
   chunk_size=1500,    # 각 청크의 최대 문자 수 (너무 크면 정보가 희석, 너무 작으면 맥락 손실)
   chunk_overlap=200,  # 인접한 청크 간 중복되는 문자 수 (맥락 연결성 유지)
   separators=[
       "\n\n",    # 빈 줄 (문단 구분 우선)
       "\n",      # 줄바꿈 (문장 구분)
       " ",       # 공백 (단어 구분)
       ".",       # 마침표 (문장 종료)
       ",",       # 쉼표 (절 구분)
       "\u200b",  # 폭 없는 공백 (웹에서 흔히 발견)
       "\uff0c",  # 전각 쉼표 (한글 텍스트용)
       "\u3001",  # 한중일 쉼표 (동아시아 언어용)
       "\uff0e",  # 전각 마침표 (한글 텍스트용)
       "\u3002",  # 한중일 마침표 (동아시아 언어용)
       "",        # 마지막 구분자 (강제 분할)
   ],
)

# 워드 문서를 로드하고 설정한 text_splitter로 분할
# Docx2txtLoader: .docx 파일을 텍스트로 변환하는 로더
loader = Docx2txtLoader('../documents/law.docx')
# 문서를 로드하면서 동시에 청크로 분할하여 리스트로 반환
document_list = loader.load_and_split(text_splitter=text_splitter)


## 6-2 UpstageEmbedding 설정방법 I

In [ ]:
# 필요한 라이브러리들을 가져온다.
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

# 환경변수 파일(.env)을 불러온다.
load_dotenv()

# 업스테이지의 SOLAR 임베딩 모델을 초기화한다.
embedding = UpstageEmbeddings(
    model='solar-embedding-1-large'  
)

## 6-3 UpstageEmbedding 설정방법 II

In [ ]:
# 필요한 라이브러리들을 가져온다.
import os

from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

# 환경변수 파일을 불러온다.
load_dotenv()
# UPSTAGE API 키를 환경변수에서 가져온다.
upstage_api_key = os.getenv("UPSTAGE_KEY")
# SOLAR 임베딩 모델을 설정한다.
embedding = UpstageEmbeddings(
    api_key=upstage_api_key,
    model="solar-embedding-1-large"
)

## 6-4 Chroma 벡터 저장소 생성

In [ ]:
# 필요한 라이브러리들을 가져온다.
from langchain_chroma import Chroma

# 데이터를 처음 저장할 때 
vector_store = Chroma.from_documents(
                    documents=document_list, 
                    embedding=embedding, 
                    collection_name='tax-original',
                    persist_directory="./tax-original"
                )


## 6-5 데이터 retrieval 테스트

In [6]:
question = '10억짜리 집을 2채 가지고 있을 때 세금을 얼마나 내나요?'

# k 값을 조절해서 얼마나 많은 데이터를 불러올지 결정
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

retriever.invoke(question)


[Document(id='e496bc88-dbec-4646-b8be-6ca0fa46f4df', metadata={'source': '../documents/law.docx'}, page_content='2. 공익법인등으로서 제1호에 해당하지 아니하는 경우: 제1항 각 호에 따른 세율\n\n3. 제1호 및 제2호 외의 경우: 다음 각 목에 따른 세율\n\n가. 2주택 이하를 소유한 경우: 1천분의 27\n\n나. 3주택 이상을 소유한 경우: 1천분의 50\n\n③주택분 과세표준 금액에 대하여 해당 과세대상 주택의 주택분 재산세로 부과된 세액(「지방세법」 제111조제3항에 따라 가감조정된 세율이 적용된 경우에는 그 세율이 적용된 세액, 같은 법 제122조에 따라 세부담 상한을 적용받은 경우에는 그 상한을 적용받은 세액을 말한다)은 주택분 종합부동산세액에서 이를 공제한다.<신설 2005. 12. 31., 2008. 12. 26., 2010. 3. 31.>\n\n④주택분 종합부동산세액을 계산할 때 주택 수 계산 및 주택분 재산세로 부과된 세액의 공제 등에 관하여 필요한 사항은 대통령령으로 정한다.<신설 2005. 12. 31., 2018. 12. 31., 2020. 6. 9.>\n\n⑤ 주택분 종합부동산세 납세의무자가 1세대 1주택자에 해당하는 경우의 주택분 종합부동산세액은 제1항ㆍ제3항 및 제4항에 따라 산출된 세액에서 제6항부터 제9항까지의 규정에 따른 1세대 1주택자에 대한 공제액을 공제한 금액으로 한다. 이 경우 제6항부터 제9항까지는 공제율 합계 100분의 80의 범위에서 중복하여 적용할 수 있다.<신설 2008. 12. 26., 2018. 12. 31., 2020. 8. 18., 2022. 9. 15.>\n\n⑥과세기준일 현재 만 60세 이상인 1세대 1주택자의 공제액은 제1항ㆍ제3항 및 제4항에 따라 산출된 세액에 다음 표에 따른 연령별 공제율을 곱한 금액으로 한다.<신설 2008. 12. 26., 2009. 5. 27., 2020. 8. 18., 202

## 6-6 retrieval_chain 생성

In [ ]:
# 필요한 라이브러리들을 가져온다.
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

llm = ChatOpenAI(model='gpt-4o-mini')

# LangChain 허브에서 검증된 RAG용 프롬프트 템플릿을 가져온다.
retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

# 검색된 문서들을 하나의 콘텍스트로 결합하는 체인을 만든다.
# stuff 방식은 모든 문서를 하나의 프롬프트에 넣는 간단한 방식이다.
combine_docs_chain = create_stuff_documents_chain(
   llm, retrieval_qa_chat_prompt
)

# retriever와 문서 결합 체인을 연결하여 최종 RAG 체인을 구성한다.
# 이제 질문을 하면 자동으로 관련 문서를 검색하고 LLM에게 전달한다.
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)


/Users/jasonkang/personal/langgraph-book/.venv/lib/python3.12/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


## 6-7 retrieval_chain을 활용한 답변 생성

In [10]:
ai_message = retrieval_chain.invoke({"input": question})
ai_message['answer']

주택에 대한 종합부동산세는 납세의무자가 소유한 주택 수에 따라 다르게 적용됩니다. 2주택 이하를 소유한 경우에는 특정 세율이 적용됩니다. 하지만 구체적인 세율은 제9조에 명시되어 있으며, 해당 세율을 적용하기 위해서는 과세표준을 계산해야 합니다.\n\n일반적으로 과세표준은 주택의 공시가격 합산에서 공제액을 뺀 금액에 공정시장가액비율을 곱한 금액으로 계산됩니다.\n\n예를 들어, 10억짜리 집을 2채 소유하고 있다면, 총 공시가격은 20억원입니다. 1세대 1주택자는 12억원이 공제되지만, 2주택 소유자는 9억원이 공제됩니다. 따라서 과세표준은 다음과 같이 계산됩니다:\n- 총 공시가격: 20억원\n- 공제액: 9억원\n- 과세표준: 20억원 - 9억원 = 11억원\n\n이후 이 과세표준에 해당하는 세율을 적용하여 세액을 계산해야 합니다. 구체적인 세율은 법령에 따라 다르므로, 해당 세율을 확인한 후 계산해야 합니다. 세율이 다르기 때문에 정확한 세액을 계산하기 위해서는 해당 세율의 정보를 추가로 제공받아야 합니다.